In [1]:
# STANDARD
from pprint import pprint

# PyPI
from bs4 import BeautifulSoup as BS
import requests
from tqdm import tqdm

# LOCAL
from interface_db import db_interface_sqlite as data
from interface_website import website_tools as tools
from interface_website import website_patheos as patheos

In [2]:
with data.database() as db:
    db.create_tables()
    db.insert_update_site_pages(1625, 149)
    #result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 47")
    #print(result[0][0])
    #for i in result:
    #    results = patheos.fetch_and_insert_blogs(i[0])
    #    print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

In [11]:
patheos.insert_website('Patheos Blogs', 'https://patheos.com/blogs')
results = patheos.fetch_and_insert_categories('Patheos Blogs')
print(results)
with data.database() as db:
    result = db.execute("SELECT name FROM categories")
    for i in result:
        results = patheos.fetch_and_insert_blogs(i[0])
        print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

Inserting Patheos Blogs - https://patheos.com/blogs
Not inserted. If exists, then https://patheos.com/blogs is already present, named Patheos Blogs.
Pulling categories for Patheos Blogs
https://www.patheos.com/buddhist-blogs
buddhist-blogs Inserted: 0 | Not Inserted: 5 | Errors: 0
https://www.patheos.com/catholic-blogs
catholic-blogs Inserted: 0 | Not Inserted: 89 | Errors: 0
https://www.patheos.com/contemplative-blogs
contemplative-blogs Inserted: 0 | Not Inserted: 8 | Errors: 0
https://www.patheos.com/evangelical-blogs
evangelical-blogs Inserted: 0 | Not Inserted: 84 | Errors: 0
https://www.patheos.com/general-christian-blogs
general-christian-blogs Inserted: 0 | Not Inserted: 18 | Errors: 0
https://www.patheos.com/hindu-blogs
hindu-blogs Inserted: 0 | Not Inserted: 4 | Errors: 0
https://www.patheos.com/jewish-blogs
jewish-blogs Inserted: 0 | Not Inserted: 6 | Errors: 0
https://www.patheos.com/latter-day-saint-blogs
latter-day-saint-blogs Inserted: 0 | Not Inserted: 17 | Errors: 0
ht

In [3]:
with data.database() as db:
    result = db.execute("SELECT * FROM blogs WHERE url = 'https://www.patheos.com/blogs/jesuscreed'")
    print(result)
    result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 149")
    print(result)

[(149, 'by Scot McKnight', 'Jesus Creed', 'https://www.patheos.com/blogs/jesuscreed', 4, '2020-04-19', 'default', '2020-04-19', 'default')]
[(1625,)]


In [4]:
import sys

number_of_new_pages = patheos.number_of_blog_pages('Jesus Creed')
print(f'\nNew Pages: {number_of_new_pages}')

1640
New Pages: 15


In [2]:
subsub_blog = requests.get('https://www.patheos.com/blogs/jesuscreed/page/2')
if subsub_blog != "<Response [404]>":
    soup2 = BS(subsub_blog.content, 'html.parser')
    for blog in soup2.find_all('h2', attrs={"class":"entry-title"}):
        blog_url1 = blog.find('a')
        blog_url2 = blog_url1['href']
        print(blog_url2)
        #subblog_list.append(blog_url2) 

https://www.patheos.com/blogs/jesuscreed/2020/01/08/reading-the-bible-fast-and-slow/
https://www.patheos.com/blogs/jesuscreed/2020/01/07/calendars-in-the-sky-rjs/
https://www.patheos.com/blogs/jesuscreed/2020/01/07/singles-myth-finding-someone-is-top-priority/
https://www.patheos.com/blogs/jesuscreed/2020/01/06/elisabeth-elliot-and-me-by-ruth-tucker/
https://www.patheos.com/blogs/jesuscreed/2020/01/06/gotta-have-faith-but-what-is-faith/
https://www.patheos.com/blogs/jesuscreed/2020/01/05/second-sunday-after-sunday/
https://www.patheos.com/blogs/jesuscreed/2020/01/04/weekly-meanderings-4-january-2020/
https://www.patheos.com/blogs/jesuscreed/2020/01/03/culture-of-storytellers-pastors-podcast/
https://www.patheos.com/blogs/jesuscreed/2020/01/02/from-the-stump-of-jesse-rjs/
https://www.patheos.com/blogs/jesuscreed/2020/01/02/myth-singles-need-to-be-pitied/


In [4]:
results_l = patheos.scrape_posts_on_page('https://www.patheos.com/blogs/jesuscreed/page/2')

for i in results_l:
    print(i)

https://www.patheos.com/blogs/jesuscreed/2020/01/08/reading-the-bible-fast-and-slow/
https://www.patheos.com/blogs/jesuscreed/2020/01/07/calendars-in-the-sky-rjs/
https://www.patheos.com/blogs/jesuscreed/2020/01/07/singles-myth-finding-someone-is-top-priority/
https://www.patheos.com/blogs/jesuscreed/2020/01/06/elisabeth-elliot-and-me-by-ruth-tucker/
https://www.patheos.com/blogs/jesuscreed/2020/01/06/gotta-have-faith-but-what-is-faith/
https://www.patheos.com/blogs/jesuscreed/2020/01/05/second-sunday-after-sunday/
https://www.patheos.com/blogs/jesuscreed/2020/01/04/weekly-meanderings-4-january-2020/
https://www.patheos.com/blogs/jesuscreed/2020/01/03/culture-of-storytellers-pastors-podcast/
https://www.patheos.com/blogs/jesuscreed/2020/01/02/from-the-stump-of-jesse-rjs/
https://www.patheos.com/blogs/jesuscreed/2020/01/02/myth-singles-need-to-be-pitied/


In [21]:
def scrape_post(post_url: str, unicode_escape_yes_no) -> object:  # post class
    invalid_urls = []
    post = data.post()
    response = requests.get(post_url)
    if response.status_code != 404:
        parsed_html = BS(response.content, 'html.parser')
        post.url = post_url
        post.title = parsed_html.find("h1", {"class" : "entry-title"}).text
        for g_basic in parsed_html.find_all("div", {"class": "main-post"}):
            post.author = g_basic.find("span", {"itemprop": "author"}).text
            post.date = g_basic.find("span", {"itemprop": "datePublished dateModified"}).text
        post.content = parsed_html.find("div", {"class": "story-block"}).text
        post.content_html
    if unicode_escape_yes_no == "yes":
        post.title = post.title.encode('unicode_escape')
        post.content = post.content.encode('unicode_escape')
    return post

In [22]:
url = 'https://www.patheos.com/blogs/jesuscreed/2020/01/02/myth-singles-need-to-be-pitied/'

post = scrape_post(url, 'no')
print(type(post))
print(post.url)
print(post.title)
print(post.author)
print(post.date)
print(post.content)

<class 'interface_db.db_interface_sqlite.post'>
https://www.patheos.com/blogs/jesuscreed/2020/01/02/myth-singles-need-to-be-pitied/
Myth: Singles Need to be Pitied
Scot McKnight
January 2, 2020


By Corey Farr
Myth: Single people should be pitied or laughed at
I have a confession to make.
When I was in middle school, my science teacher’s name was Mr. Tarullo. Mr. T was in his mid-50’s, thin, bald, a bit quirky, and single. He was a loving man, a great teacher, and extremely dedicated to his work. He was always one of the first to get to the school in the morning and one of the last to leave in the evening. When he wasn’t going over tests and quizzes, he was usually reading his Bible in silence in his classroom, in that funny way where he’d put his elbows on his desk and cup his bald forehead in his palms as he stared down at the pages. He was close to his family and his church community. He would drive several hours up to visit his widowed mother in a little town up north to help her m

In [ ]:
def fetch_and_insert_posts(blog_name: str) -> tools.insert_results():
    pass